In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, QuatE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_0_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_0_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_0_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [5]:
model_name = 'quatE_0_transductive'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [8]:
model = QuatE(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 5913600
Space occupied: 23654400 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory quatE_0_transductive created successfully!


In [10]:
learning_rate = 5e-2 #1e-3 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 4.530369281768799
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0010693481946845856
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0032081882317982658
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.002138768213241426
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.001069348189048469
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.003208188572898507
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0021387680899351835
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0010693481946845856
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0032081882317982658
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.002138768213241426
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0015564202334630351
Metric: validation.both.optimistic.hits_at_1 = 0.000778210116

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 4.535050392150879
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0011380417909718445
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0018088205837276917
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0014734311873497677
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0011380418436601758
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0018088207580149172
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0014734312426298857
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0011380417909718445
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00180882057486637
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0014734311829191073
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0003891050583657588
Metric: validation.both.optimistic.hits_at_1 = 0.0001945525

...

Step: 60
Metric: loss = 0.1048138439655304
Step: 60
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.040589329884228094
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.07968797455799759
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.060138652221112844
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.040589332580566406
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.07968797534704208
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.06013865023851394
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.040589327725649584
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.07968797439210273
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.060138651058876155
Metric: validation.head.optimistic.hits_at_1 = 0.017509727626459144
Metric: validation.tail.optimistic.hits_at_1 = 0.04357976653696498
Metric: validation.both.optimistic.hits_at_1 = 0.030

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 61
Metric: loss = 0.09980110079050064
Step: 61
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.04126613779306252
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.07932069111208623
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.06029341445257437
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.04126613587141037
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.07932069897651672
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.06029341369867325
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.04126613779306252
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0793206909859701
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.06029341438951631
Metric: validation.head.optimistic.hits_at_1 = 0.01867704280155642
Metric: validation.tail.optimistic.hits_at_1 = 0.042412451361867706
Metric: validation.both.optimistic.hits_at_1 = 0.03054474

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 62
Metric: loss = 0.10105881094932556
Step: 62
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.04110302439091684
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.07856956348718153
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.059836293939049184
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.041103024035692215
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.07856956124305725
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.059836294502019875
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.041103024354585876
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.07856956348718153
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0598362939208837
Metric: validation.head.optimistic.hits_at_1 = 0.01867704280155642
Metric: validation.tail.optimistic.hits_at_1 = 0.038910505836575876
Metric: validation.both.optimistic.hits_at_1 = 0.0287

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
           0
0   4.530369
1   4.535050
2   4.535162
3   4.535879
4   4.534874
..       ...
57  0.097804
58  0.105785
59  0.104814
60  0.099801
61  0.101059

[62 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/10.4k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.884650   0.884650     0.884650
hits_at_1                     0.840478   0.840478     0.840478
hits_at_3                     0.912340   0.912340     0.912340
hits_at_5                     0.941624   0.941624     0.941624
hits_at_10                    0.972931   0.972931     0.972931
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.937273   0.937273     0.937273
hits_at_1                     0.918890   0.918890     0.918890
hits_at_3                     0.947019   0.947019     0.947019
hits_at_5                     0.956363   0.956363     0.956363
hits_at_10                    0.972353   0.972353     0.972353
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.910961   0.910961     0.910961
hits_at_1                     0.879684   0.879684     0.879684
hits_at_3                     0.929679  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/2.57k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.047336   0.047336     0.047336
hits_at_1                     0.024125   0.024125     0.024125
hits_at_3                     0.047471   0.047471     0.047471
hits_at_5                     0.056809   0.056809     0.056809
hits_at_10                    0.083658   0.083658     0.083658
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.079764   0.079764     0.079764
hits_at_1                     0.040856   0.040856     0.040856
hits_at_3                     0.080934   0.080934     0.080934
hits_at_5                     0.103891   0.103891     0.103891
hits_at_10                    0.142412   0.142412     0.142412
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.063550   0.063550     0.063550
hits_at_1                     0.032490   0.032490     0.032490
hits_at_3                     0.064202  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/3.27k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.040565   0.040565     0.040565
hits_at_1                     0.018349   0.018349     0.018349
hits_at_3                     0.034862   0.034862     0.034862
hits_at_5                     0.048624   0.048624     0.048624
hits_at_10                    0.077676   0.077676     0.077676
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.073729   0.073729     0.073729
hits_at_1                     0.035474   0.035474     0.035474
hits_at_3                     0.067278   0.067278     0.067278
hits_at_5                     0.095719   0.095719     0.095719
hits_at_10                    0.144648   0.144648     0.144648
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.057147   0.057147     0.057147
hits_at_1                     0.026911   0.026911     0.026911
hits_at_3                     0.051070  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   

                                                value  
0                                              cuda:0  
1                                            236544

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)